In [ ]:
!pip install python-geohash
!pip install geopandas
!pip install polygon_geohasher
!pip install geojson

  Created wheel for python-geohash: filename=python_geohash-0.8.5-cp36-cp36m-linux_x86_64.whl size=43665 sha256=253fc7ba99ff10e9eace0baddeb3503edfab04f40e24755b643605cbc8832dd5
  Stored in directory: /root/.cache/pip/wheels/b5/64/5a/6a286481fc7c2a698d2f297d4c90af19946be430b23eba9a33
Successfully built python-geohash
     |████████████████████████████████| 962kB 7.6MB/s 
     |████████████████████████████████| 10.9MB 1.5MB/s 
     |████████████████████████████████| 14.7MB 209kB/s 
  Created wheel for polygon-geohasher: filename=polygon_geohasher-0.0.1-cp36-none-any.whl size=4140 sha256=da3c01d90fe5a8a7cb69d9e8575ba8f87c78d17d93cae97dc78e2308ae1dd205
  Stored in directory: /root/.cache/pip/wheels/d2/f2/58/66d0dbbb82fac601389214faa50cb6f8ebcf494421dafdec52
Successfully built polygon-geohasher


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import pandas as pd 
import numpy as np
import geohash
import folium
from polygon_geohasher.polygon_geohasher import geohash_to_polygon
import geojson as json 
import numpy as np
import geopandas as gpd
import pandas as pd
import collections
import statistics
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import LambdaLR
from torch.autograd import Variable
_base32 = '0123456789bcdefghjkmnpqrstuvwxyz'
_base32_map = {}
for i in range(len(_base32)):
	_base32_map[_base32[i]] = i
del i

In [ ]:
class Args:
    def __init__(self):
        self.cuda = True
        self.no_cuda = False
        self.seed = 1
        self.batch_size = 64
        self.test_batch_size = 1000
        self.epochs = 10
        self.lr = 0.01
        self.momentum = 0.5
        self.log_interval = 10
        self.img_size=9
        self.num_filtre=64
        self.size_filtre=5
        self.kernel_maxpooling=2
        self.stride_maxpooling=2
        self.output_size_linear=64
        self.hidden_size=16
        self.output_size_linear_lstm=1
        self.batsh_size=15
        self.seq_len=8
        self.date_rng = pd.to_datetime(pd.date_range(start='2018-01-01 00:00:00', end='2019-12-30 00:00:00', freq='H'))
        self.number_of_zone_training=15

args = Args()

In [ ]:
df=pd.read_csv('/content/drive/My Drive/bq-results-20200627-184642-1olk9ybv9nci/bq-results-20200627-184642-1olk9ybv9nci.csv')

In [ ]:
Weather=pd.read_excel('/content/drive/My Drive/Data/Weather_Data.xls')

In [ ]:
def Weather_cleaning(Weather) :
  Weather['Time']=pd.to_datetime(Weather['Time'])
  Weather['Time']=Weather['Time'].dt.strftime('%Y-%m-%d %H:00:00')
  Data_Weather=pd.pivot_table(Weather, values=['T','U','Ff'], index=['Time'], aggfunc=np.mean)
  Data_Weather['Time'] =Data_Weather.index
  Data_Weather['Time']=pd.to_datetime(Data_Weather['Time'])
  #Data_Weather['day_of_week'] = Data_Weather['Time'].dt.day_name()
  Data_Weather['hour_of_day'] = Data_Weather['Time'].dt.hour
  Data_Weather=Data_Weather.drop(['Time'],axis=1)
  dfDummies = pd.get_dummies(Data_Weather['hour_of_day'], prefix = 'HOUR')
  #dfDummies = pd.get_dummies(Data_Weather['day_of_week'], prefix = 'category')
  Data_Weather=Data_Weather.drop(['hour_of_day'],axis=1)
  #Data_Weather=Data_Weather.drop(['day_of_week'],axis=1)
  #df=Data_Weather
  df = pd.concat([Data_Weather, dfDummies], axis=1)
  #df=df.reindex(['T','U','Ff','hour_of_day','category_Sunday','category_Monday','category_Thursday','category_Tuesday','category_Wednesday','category_Saturday'],axis=1)
  df=df.rename(columns={"T": "temperature_of_the_day", "U": "Humidity",'Ff':'Wind_speed'})
  df=df.drop(["temperature_of_the_day","Humidity"],axis=1)
  df = df.loc[(df.index >= str(args.date_rng.min())) & (df.index <= str(args.date_rng.max()))]
  df.index = pd.DatetimeIndex(df.index)
  df= df.reindex(args.date_rng, fill_value=0)
  df=df.fillna(0 )
 
  return df


<h2>geohashFunction</h2>
<h3> Description: </h3>
La fonction <b>geohashFunction</b> reçoit en entrée les coordonnée d'un point et la résolution demandé et retourne le code géohash qui lui corréspond: 
<h3> Paramètres: </h3>
<ul>
  <li>X: latitude</li>
  <li>Y: Longitude</li>
  <li>resl: La résolution</li>
</ul> 
<h3> Valeur retournée: </h3>
<ul>
   <li>Code géohash</li>
</ul> 




In [ ]:
def geohashFunction(x,y):
   return geohash.encode(x,y,6)

<h2>decode_c2i</h2>
<h3> Description: </h3>
La fonction <b>decode_c2i</b> reçoit le hash code d'une 
zone et retourne les corrdénnées du centre de la zone.
<h3> Paramètres: </h3>
<ul>
  <li>hashcode: le hashcode de la zone</li>
</ul> 
<h3> Valeur retournée: </h3>
<ul>
   <li>tuple contenant les cordonnées</li>
</ul> 

In [ ]:
def decode_c2i(hashcode):
	lon = 0
	lat = 0
	bit_length = 0
	lat_length = 0
	lon_length = 0
	for i in hashcode:
		t = _base32_map[i]
		if bit_length%2==0:
			lon = lon<<3
			lat = lat<<2
			lon += (t>>2)&4
			lat += (t>>2)&2
			lon += (t>>1)&2
			lat += (t>>1)&1
			lon += t&1
			lon_length+=3
			lat_length+=2
		else:
			lon = lon<<2
			lat = lat<<3
			lat += (t>>2)&4
			lon += (t>>2)&2
			lat += (t>>1)&2
			lon += (t>>1)&1
			lat += t&1
			lon_length+=2
			lat_length+=3
		
		bit_length+=5
	
	return (lat,lon,lat_length,lon_length)


In [ ]:
def encode_i2c(lat,lon,lat_length,lon_length):
	precision = int((lat_length+lon_length)/5)
	if lat_length < lon_length:
		a = lon
		b = lat
	else:
		a = lat
		b = lon
	
	boost = (0,1,4,5,16,17,20,21)
	ret = ''
	for i in range(precision):
		ret+=_base32[(boost[a&7]+(boost[b&3]<<1))&0x1F]
		t = a>>3
		a = b>>2
		b = t
	
	return ret[::-1]

<h2>neighbors</h2>
<h3> Description: </h3>
La fonction <b>neighbors</b> reçoit en entrée le hashcode d'une zone et la taille du quartier et retourne une matrice de hashcodes.
Au milieu de cette matrice il y a le hashcode en entrée et les autres cases de la matrice contient les hashcodes des zones voisines.
La matrice est carré d'une taille (S x S).
S doit être impair.
<h3> Paramètres: </h3>
<ul>
  <li>hashcode: le hashcode de la zone</li>
  <li>S: La taille du quartier</li>
</ul> 
<h3> Valeur retournée: </h3>
<ul>
   <li>Une matrice hashcodes</li>
</ul> 

In [ ]:
def neighbors(hashcode,S):

	
	(lat,lon,lat_length,lon_length) = decode_c2i(hashcode)
	ret = []
	rang=S // 2
	tab=[]
	tab2=[]
	tab3=[]

	for i in range (1,rang+1):
							tab.append(lat+rang+1-i)
							tab3.append(lat-i)       
	for i in range(-rang,rang+1) :
     					tab2.append(lon+i)
 
	for tlat in tab:
          if not tlat >> lat_length:
            for tlon in tab2:
              ret.append(encode_i2c(tlat,tlon,lat_length,lon_length))
        
	tlat = lat
	for tlon in tab2:
		code = encode_i2c(tlat,tlon,lat_length,lon_length) 
		if code:
			ret.append(code)
   
	for tlat in tab3:
          if tlat >= 0:
            for tlon in tab2:
              ret.append(encode_i2c(tlat,tlon,lat_length,lon_length))
          
	return np.array(ret).reshape(S,S)

In [ ]:
def clean_data(df) :
  data=df.loc[(df['requested_date'] >= str(args.date_rng.min())) & (df['requested_date']<= str(args.date_rng.max()))]
  data=df[['p_lat','p_lng','requested_date','coupon']]
  data['geohash']=data.apply(lambda x: geohashFunction(x.p_lat, x.p_lng), axis=1)
  data['geometry']=data['geohash'].apply(geohash_to_polygon)
  data['requested_date']=pd.to_datetime(data['requested_date'])
  data['requested_date']=data['requested_date'].dt.strftime('%Y-%m-%d %H:00:00')
  return data

In [ ]:
dfcoupon=df[['requested_date','coupon']]
dfcoupon=dfcoupon.dropna()
dfcoupon['requested_date']=pd.to_datetime(dfcoupon['requested_date'])
dfcoupon['requested_date']=dfcoupon['requested_date'].dt.strftime('%Y-%m-%d %H:00:00')
pd.DataFrame({'requested_date':dfcoupon['requested_date'].unique()})

,requested_date
0,2019-12-11 13:00:00
1,2019-11-28 19:00:00
2,2019-12-10 13:00:00
3,2019-10-13 07:00:00
4,2019-01-24 19:00:00
...,...
8246,2019-11-17 00:00:00
8247,2019-09-23 12:00:00
8248,2019-08-12 13:00:00
8249,2018-08-03 04:00:00


<h2>seq_of_demand_zone</h2>
<h3> Description: </h3>
La fonction <b>seq_of_demand_zone</b> construit pour chaque timestemp une matrice contenant les valeurs de la demande pour chaqu'une des zones appartenant au voisinage de la zone en entrée, elle retourne donc l'état du quartier de la zone en entrée pour chaque timestemp.
<h3> Paramètres: </h3>
<ul>
  <li>data: une dataframe contenant les données de la demande pour chaque zone.</li>
  <li>zone: le géohash de la zone.</li>
  <li>reshape:le nombre de timestemps.</li>
</ul> 
<h3> Valeur retournée: </h3>
<ul>
   <li>Un tensor de quatre dimension, (nombre de timestemps, 1,S,S), ou S et la taille du quartier.</li>
</ul> 

In [ ]:
def seq_of_demand_zone(data,zone,reshape) :
   x=[]     
   Data=pd.crosstab(data['requested_date'],data['geohash'])
   Data.index = pd.DatetimeIndex(Data.index)
   Data= Data.reindex(args.date_rng, fill_value=0)
   for zn in neighbors(zone,args.img_size).reshape(args.img_size*args.img_size,)  :
     if zn not in Data.columns: 
        print(zn)
        Data[str(zn)]= 0       
   Data=Data[neighbors(zone,args.img_size).reshape(args.img_size*args.img_size,)]   
   Data= Data.reindex(neighbors(zone,args.img_size).reshape(args.img_size*args.img_size,), axis=1)                     
   for i in Data.index:
        x.append(torch.from_numpy(np.array(Data.loc[i]).reshape(args.img_size,args.img_size)))                                                       
   tensor= torch.stack(x)
   return tensor.reshape(reshape,1,args.img_size,args.img_size)

<h2>create_inout_sequences</h2>
<h3> Description: </h3>
La fonction <b>create_inout_sequences</b> reçoit en entrée un tensor de quatre dimension, (nombre de timestemps, 1,S,S), ou S et la taille d'un quartier (c'est le résultat de la fonction <b>seq_of_demand_zone</b> ) et la taille d'une séquence, elle construit des séquence d'une taille <b>tw</b> et retourne une list de tuplet, tel que chaque tuplet contient deux valeurs la première et un tensor de dimension trois (tw: taille de la séquence, S: la taille du quartier,S), la deuxième valeur contient la valeur de la demande de la zone en question dans le timestemp qui suit celle de la séquence construit.
<h3> Paramètres: </h3>
<ul>
  <li>input_data: un tensor de dimension quatre.</li>
  <li>tw: la taille de la séquence</li>
</ul> 
<h3> Valeur retournée: </h3>
<ul>
   <li>une list de tuplet.</li>
</ul> 

In [ ]:
def create_inout_sequences(input_data, tw):
    inout_seq = []
    L = len(input_data)
    for i in range(L-tw):
        train_seq = input_data[i:i+tw]
        train_label = input_data[i+tw:i+tw+1]
        inout_seq.append((train_seq ,train_label[0][0][args.img_size//2][args.img_size//2]))
    return inout_seq

<h2>create_inout_sequences_extarnal_data</h2>
<h3> Description: </h3>
La fonction <b>create_inout_sequences_extarnal_data</b> reçoit en entrée les données du climat et la langueure de la séquence, elle retourne un tensor de dimension trois, contenant pour chaque séquence de données les données du climat qui lui corresponds.
<h3> Paramètres: </h3>
<ul>
  <li>Une dataframe contenant les données du climat</li>
  <li>La langueure de la séquence</li>
</ul> 
<h3> Valeur retournée: </h3>
<ul>
   <li>Un tensor de dimmension trois, contenant les données de climat de chaque séquence.</li>

</ul> 

In [ ]:
def create_inout_sequences_extarnal_data(input_data, tw):
    inout_seq_external_data = []
    L = len(input_data)
    for i in range(L-tw):
        train_seq_external_data = input_data[i:i+tw]
        train_label = input_data[i+tw:i+tw+1]
        inout_seq_external_data.append(train_seq_external_data.values)   
    return torch.FloatTensor(inout_seq_external_data).view(L-tw ,args.seq_len,len(input_data.columns))

<h2>create_data_final</h2>
<h3> Description: </h3>
La fonction <b>create_data_final</b> combine les fonctions <b>create_inout_sequences</b> et <b>seq_of_demand_zone</b> pour créer les séquence de données pour entraîner le model.
<h3> Paramètres: </h3>
<ul>
  <li>Une dataframe contenant les données de la demande</li>
  <li>Le nombre de zone sur lesquelles on entraine le modèle</li>
  <li>La langueure de la séquence</li>
</ul> 
<h3> Valeur retournée: </h3>
<ul>
   <li>une list de tuplet.</li>
</ul>

In [ ]:
def create_data_final(data,number_of_zone,sequence_len):
  sequence_data=[]
  Data=pd.crosstab(data['requested_date'],data['geohash'])
  Data.index = pd.DatetimeIndex(Data.index)
  Data=Data.reindex(Data.mean().sort_values(ascending=False).index, axis=1).iloc[:,0:args.number_of_zone_training]
  for zone in Data.columns:
    print(zone)
    sequence_data.append(create_inout_sequences(seq_of_demand_zone(data,zone,len(args.date_rng)),sequence_len))
  return sequence_data

In [ ]:
data= clean_data(df)  
External_Feautre=Weather_cleaning(Weather)
External_Feautre_sequence=create_inout_sequences_extarnal_data(External_Feautre,args.seq_len)
seq=create_data_final(data,args.number_of_zone_training,args.seq_len)

In [ ]:
len(External_Feautre_sequence)

In [ ]:
#model DMLVST_DATA
class RMSELoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss()
        
    def forward(self,y_hats,y):
        return torch.sum((y-y_hats)**2)
        #return np.sqrt(torch.sum((y-y_hats)))
        #return self.mse(yhat,y)

args.cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)

kwargs = {'num_workers': 1, 'pin_memory': True} if args.cuda else {}



class CNN(nn.Module):
    def __init__(self,img_size,num_filtre,size_filtre,kernel_maxpooling,stride_maxpooling,output_size_linear):
        super(CNN, self).__init__()
        self.num_filtre=num_filtre
        self.conv1 = nn.Conv2d(1,num_filtre, kernel_size=size_filtre)
        self.conv_out=(img_size-size_filtre)+1
        self.max_pool=nn.MaxPool2d( kernel_size=kernel_maxpooling,stride = stride_maxpooling)
        self.max_pool_out=(self.conv_out-kernel_maxpooling)//stride_maxpooling+1
        self.fc1 = nn.Linear(num_filtre*self.max_pool_out*self.max_pool_out,output_size_linear)
    

    def forward(self, x):
        x=self.conv1(x)
        x=self.max_pool(x)
        x=F.relu(x)
        x=F.dropout(x,0.3,training=True)
        x=x.view(-1,self.num_filtre*self.max_pool_out*self.max_pool_out)
        x=self.fc1(x)
        return x


class Combine(nn.Module):
    def __init__(self,img_size,num_filtre,size_filtre,kernel_maxpooling,stride_maxpooling,output_size_linear,hidden_size,output_size_linear_lstm,batsh_size,seq_len):
        super(Combine, self).__init__()
        self.batsh_size=batsh_size
        self.output_size_linear=output_size_linear
        self.seq_len=seq_len
        self.hidden_size=hidden_size
        self.cnn = CNN(img_size,num_filtre,size_filtre,kernel_maxpooling,stride_maxpooling,output_size_linear)
        #self.num_filtre=num_filtre
        #self.conv1 = nn.Conv2d(1,num_filtre, kernel_size=size_filtre)
        #self.conv_out=(img_size-size_filtre)+1
        #self.max_pool=nn.MaxPool2d( kernel_size=kernel_maxpooling,stride = stride_maxpooling)
        #self.max_pool_out=(self.conv_out-kernel_maxpooling)//stride_maxpooling+1
        #self.fc1 = nn.Linear(num_filtre*self.max_pool_out*self.max_pool_out,output_size_linear)


        self.lstm = nn.LSTM(output_size_linear+25,self.hidden_size)

        self.linear = nn.Linear(self.hidden_size,output_size_linear_lstm)

        self.hidden_cell = (torch.zeros(1,self.batsh_size,self.hidden_size),
                            torch.zeros(1,self.batsh_size,self.hidden_size))
    
       
    def forward(self, x,x_external_data):
        c_out = self.cnn(x)
        c_out=c_out.view(self.seq_len,self.batsh_size,self.output_size_linear )
        ex=(x_external_data).view(args.seq_len,1,25)
        data_external_for_reshape=[]
        for i in range (args.number_of_zone_training) :
             data_external_for_reshape.append(ex)
        data_external_for_reshape=torch.cat(data_external_for_reshape).view(args.seq_len,args.number_of_zone_training,25)
        #print(data_external_for_reshape)
        input_lstm = torch.cat([c_out,data_external_for_reshape], 2)
        #x=self.conv1(x)
        #x=self.max_pool(x)
        #x=F.relu(x)
        #x=x.view(-1,self.num_filtre*self.max_pool_out*self.max_pool_out)
        #c_out=F.normalize(self.fc1(x))
        #input_lstm=c_out.view(self.seq_len,self.batsh_size,self.output_size_linear )
        lstm_out, self.hidden_cell = self.lstm(input_lstm, self.hidden_cell)
        predictions = self.linear(self.hidden_cell[0].view(self.batsh_size, -1))
      
        return  predictions 





 


In [ ]:
use_cuda=True

In [ ]:
model=Combine(args.img_size,args.num_filtre,args.size_filtre,args.kernel_maxpooling,args.stride_maxpooling,args.output_size_linear,args.hidden_size,args.output_size_linear_lstm,args.batsh_size,args.seq_len)
#torch.save(model.state_dict(),'model.pt')
model.load_state_dict(torch.load('/content/drive/My Drive/Data/model.pt'))
model=model.eval()
model


Combine(
  (cnn): CNN(
    (conv1): Conv2d(1, 64, kernel_size=(5, 5), stride=(1, 1))
    (max_pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (fc1): Linear(in_features=256, out_features=64, bias=True)
  )
  (lstm): LSTM(89, 16)
  (linear): Linear(in_features=16, out_features=1, bias=True)
)

In [ ]:
model=Combine(args.img_size,args.num_filtre,args.size_filtre,args.kernel_maxpooling,args.stride_maxpooling,args.output_size_linear,args.hidden_size,args.output_size_linear_lstm,args.batsh_size,args.seq_len)
criterion =RMSELoss()
#criterion = nn.MSELoss()
#optimizer = optim.SGD(model.parameters(), lr=0.008 , momentum=args.momentum)
optimizer = torch.optim.Adam(model.parameters(), lr=0.00008 )

for epoch in range (50) :
   total_loss = []
   model.train()
   for i in range(0,16000)  :
        input_data=[]
        y=[]
        for j in range(args.number_of_zone_training) :
            input_data.append(seq[j][i][0])
            y.append(seq[j][i][1])
        if use_cuda and torch.cuda.is_available():   
            x_train_external_data=External_Feautre_sequence[i]
            x_train_zones_seq=torch.cat(input_data)
        y_train_zones_seq=torch.from_numpy(np.array(y)).view(args.number_of_zone_training,-1) 
        model.hidden_cell[0].detach_()
        model.hidden_cell[1].detach_()
        optimizer.zero_grad()
        output = model(x_train_zones_seq.float(),x_train_external_data.float())
        loss  = criterion(output, y_train_zones_seq.float())#torch.sqrt(criterion(output, y_train_zones_seq.float()))
        loss.backward(retain_graph=False)
        optimizer.step()
        total_loss.append(loss.item())
    #*******************************Validation******************************************
   pred=[]
   total_loss_val = []
   plotpred=[]
   ploty=[]
   model.eval()
   for i in range(16000,17400)  :
          input_data=[]
          y=[]
          for j in range(args.number_of_zone_training) : 
              input_data.append(seq[j][i][0])
              y.append(seq[j][i][1])
          if use_cuda and torch.cuda.is_available():  
              x_train_external_data=External_Feautre_sequence[i]    
              x_train_zones_seq=torch.cat(input_data)
              y_train_zones_seq=torch.from_numpy(np.array(y)).view(args.number_of_zone_training,-1)
          pred=model(x_train_zones_seq.float(),x_train_external_data.float())
          plotpred.append(pred.detach().numpy().reshape(args.number_of_zone_training))
          ploty.append(np.array(y))
          loss  =criterion(pred, y_train_zones_seq.float())#torch.sqrt( criterion(pred, y_train_zones_seq.float()))
          total_loss_val.append(loss.item())


   #print('sequence : ',i,'Avrage sequences loss :',(sum(total_loss)/len(total_loss)), 'The loss of the last sequence :', loss.item())    
   if epoch % 1 == 0:
           print('epoch: ',epoch,'Avrage sequences loss :',(sum(total_loss)/len(total_loss)),epoch,'Avrage sequences loss validation :',(sum(total_loss_val)/len(total_loss_val)))     

In [ ]:
pred=[]
total_loss_val = []
plotpred=[]
ploty=[]
model.eval()
for i in range(16000,17400)  :
          input_data=[]
          y=[]
          for j in range(args.number_of_zone_training) : 
              input_data.append(seq[j][i][0])
              y.append(seq[j][i][1])
          if use_cuda and torch.cuda.is_available():  
              x_train_external_data=External_Feautre_sequence[i]    
              x_train_zones_seq=torch.cat(input_data)
              y_train_zones_seq=torch.from_numpy(np.array(y)).view(args.number_of_zone_training,-1)
          pred=model(x_train_zones_seq.float(),x_train_external_data.float())
          plotpred.append(pred.detach().numpy().reshape(args.number_of_zone_training))
          ploty.append(np.array(y))
          loss  =criterion(pred, y_train_zones_seq.float())#torch.sqrt( criterion(pred, y_train_zones_seq.float()))
          total_loss_val.append(loss.item())


In [ ]:
ploty=pd.DataFrame(data=ploty)


In [ ]:
plotpred=pd.DataFrame(data=plotpred)
plotpred[plotpred<1]=0

In [ ]:
plotpred

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,4.369356,2.787366,2.316286,2.669726,2.120655,1.288558,1.859458,2.338140,1.782753,1.520971,1.092058,1.83819,1.050179,1.388151,2.231528
1,3.295418,1.850683,1.768195,0.000000,1.133864,0.000000,0.000000,2.858488,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,1.091487
2,0.000000,1.073386,0.000000,0.000000,0.000000,1.196344,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,1.321635
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1395,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.014826,0.00000,0.000000,0.000000,0.000000
1396,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
1397,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
1398,0.000000,0.000000,0.000000,0.000000,1.472413,1.180345,0.000000,1.173906,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000


In [ ]:
ploty

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,4,5,0,0,0,1,0,5,0,1,3,4,0,4,1
1,1,0,2,1,0,0,0,0,0,0,3,1,1,0,3
2,0,2,0,0,1,0,0,0,0,0,1,1,0,0,1
3,2,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1395,4,5,0,0,0,2,0,0,0,0,0,2,1,1,2
1396,1,0,4,1,1,3,0,1,0,0,0,4,0,0,0
1397,2,1,0,0,0,2,2,1,0,0,0,0,0,0,1
1398,2,2,0,0,1,3,0,0,0,0,1,0,2,0,3


In [ ]:
for i in range(5):
  df_all = pd.merge(ploty[i],plotpred[i] , how = 'outer', left_index=True, right_index=True)
  df_all.plot(figsize=(200,40))

NameError: ignored

In [ ]:
def mapFolium (data,values):
  Data=pd.crosstab(data['requested_date'],data['geohash'])
  Data.index = pd.DatetimeIndex(Data.index)
  Data=Data.reindex(Data.mean().sort_values(ascending=False).index, axis=1).iloc[:,0:args.number_of_zone_training]
  dataFolium=pd.DataFrame({'geohash':Data.columns,'value':values})
  dataFolium['geometry']= dataFolium['geohash'].apply(geohash_to_polygon)
  dataFolium=gpd.GeoDataFrame(dataFolium)
  dataFolium.crs = {'init': 'epsg:4326'}
  lat, lng = (36.7525000,3.0419700)
  m = folium.Map((lat, lng), zoom_start=12)
  folium.Choropleth(geo_data=dataFolium, 
                  name='choropleth',
                  data=dataFolium,
                  columns=['geohash', 'value'],
                  key_on='feature.properties.geohash',
                  fill_color='YlGn',
                  fill_opacity=0.7,
                  line_opacity=0.2,
                  legend_name='asdf').add_to(m)
  return m                

In [ ]:
value=[val for val in plotpred.iloc[0,:]]
datafol=data
mapFolium(datafol,value)

In [ ]:
len(value)

In [ ]:
data

In [ ]:
def train(epoch):
    
    for i in range(1128)  :
        input_data=[]
        y=[]
        for j in range(number_of_zone_training) :
            input_data.append(seq[j][i][0])
            y.append(seq[j][i][1])
        External_Feautre_sequence  
        #x_train_external_feautre=External_Feautre_sequence[i]  
        x_train_zones_seq=torch.cat(input_data)
        y_train_zones_seq=torch.from_numpy(np.array(y)).view(number_of_zone_training,-1)
        #x_train_zones_seq= torch.FloatTensor(x_train_zones_seq)
       
        #model.hidden_cell = (torch.zeros(1,number_of_zone_training, model.hidden_size),
                       #torch.zeros(1, number_of_zone_training,model.hidden_size)) 
        model.hidden_cell[0].detach_()
        model.hidden_cell[1].detach_()
        optimizer.zero_grad()
        output = model(x_train_zones_seq.float())
        loss  = criterion(output, y_train_zones_seq.float())
        loss.backward(retain_graph=False)
        optimizer.step()
        total_loss.append(loss.item())
    print('sequence : ',i,'Avrage sequences loss :',(sum(total_loss)/len(total_loss)), 'The loss of the last sequence :', loss.item())    
    if epoch % 3 == 0:
           print('epoch: ',epoch,'Avrage sequences loss :',(sum(total_loss)/len(total_loss)), 'The loss of the last sequence :', loss.item())

In [ ]:
np.array(y)



<h3><b>Feature selection</b> </h3>





In [ ]:
External_Feautre=Weather_cleaning(Weather)

In [ ]:
Data.shape

In [ ]:
  Weather['Time']=pd.to_datetime(Weather['Time'])
  Weather['Time']=Weather['Time'].dt.strftime('%Y-%m-%d %H:00:00')
  Data_Weather=pd.pivot_table(Weather, values=['T','U','Ff'], index=['Time'], aggfunc=np.mean)
  Data_Weather['Time'] =Data_Weather.index
  Data_Weather['Time']=pd.to_datetime(Data_Weather['Time'])
  Data_Weather['day_of_week'] = Data_Weather['Time'].dt.day_name()
  Data_Weather['hour_of_day'] = Data_Weather['Time'].dt.hour
  Data_Weather=Data_Weather.drop(['Time'],axis=1)
  df=Data_Weather
  #dfDummies = pd.get_dummies(Data_Weather['day_of_week'], prefix = 'category')
  #Data_Weather=Data_Weather.drop(['day_of_week'],axis=1)
  #df = pd.concat([Data_Weather, dfDummies], axis=1)
  #df=df.reindex(['T','U','Ff','hour_of_day','category_Sunday','category_Monday','category_Thursday','category_Tuesday','category_Wednesday','category_Saturday'],axis=1)
  #df=df.rename(columns={"T": "temperature_of_the_day", "U": "Humidity",'Ff':'Wind_speed'})
  df = df.loc[(df.index >= str(args.date_rng.min())) & (df.index <= str(args.date_rng.max()))]
  df.index = pd.DatetimeIndex(df.index)
  df= df.reindex(args.date_rng, fill_value=0)
  df=df.fillna(0)
  
 

In [ ]:
from sklearn import preprocessing
df['day_of_week']=df['day_of_week'].astype(str)
le = preprocessing.LabelEncoder()
le.fit(df['day_of_week'])
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
df['day_of_week']=le.transform(df['day_of_week'])

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn import preprocessing
from numpy import set_printoptions
import matplotlib.pyplot as plt
# load data
X= df

for zone in Data.columns:
    Y = Data[zone]
    # feature extraction
    test = SelectKBest(score_func=f_classif, k=3)
    fit = test.fit(X, Y)
    # summarize scores
    set_printoptions(precision=3)
    #print(fit.scores_.argmax())
   #features = fit.transform(X)
    # summarize selected features
    #features
    plt.rcParams["figure.figsize"] = (20,10)
    plt.plot([0,1,2,3,4],fit.scores_)
    

In [ ]:
df